<a href="https://colab.research.google.com/github/ccwu0918/classroom-hahow-sqlfifty/blob/main/04-calculated-fields/lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL 的五十道練習

> 衍生計算欄位

[數據交點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
!git clone https://github.com/ccwu0918/classroom-hahow-sqlfifty.git

Cloning into 'classroom-hahow-sqlfifty'...
remote: Enumerating objects: 915, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 915 (delta 89), reused 70 (delta 29), pack-reused 749
Receiving objects: 100% (915/915), 68.99 MiB | 11.46 MiB/s, done.
Resolving deltas: 100% (581/581), done.
Updating files: 100% (130/130), done.


In [2]:
%cd classroom-hahow-sqlfifty

/content/classroom-hahow-sqlfifty


In [3]:
%load_ext sql

In [4]:
%sql sqlite:///./databases/imdb.db
%sql ATTACH './databases/covid19.db' AS covid19
%sql ATTACH './databases/twElection2020.db' AS twElection2020
%sql ATTACH './databases/nba.db' AS nba

 * sqlite:///./databases/imdb.db
Done.
 * sqlite:///./databases/imdb.db
Done.
 * sqlite:///./databases/imdb.db
Done.


[]

## 單獨存在的 `SELECT`

## 在沒有 `FROM` 的情況下，`SELECT` 也能夠單獨存在於一段 SQL 之中

In [6]:
%%sql
SELECT 'Hello, SQL!' AS hello_sql; -- 單獨存在的 SELECT

 * sqlite:///./databases/imdb.db
Done.


hello_sql
"Hello, SQL!"


## 在 `SELECT` 後面除了指定資料表的欄位，也能夠擺放常數，常見的常數類型有：

- 整數（INTEGER）。
- 帶小數點的浮點數（REAL）。
- 用單引號或雙引號標記的文字（TEXT）。
- 空值、遺漏值（NULL）。

## 我們使用單獨存在的 `SELECT` 將常見的常數類型顯示出來

In [7]:
%%sql
SELECT 5566 AS an_integer,
       5566.0 AS a_real,
       'Hello, SQL!' AS a_text,
       NULL AS a_null;

 * sqlite:///./databases/imdb.db
Done.


an_integer,a_real,a_text,a_null
5566,5566.0,"Hello, SQL!",None


## 這表示即便在資料表中沒有儲存常數，依然能在 SQL 中加入這些常數

In [10]:
%%sql
SELECT heightFeet,
       30.48 AS foot_to_cm,
       heightInches,
       2.54 AS inch_to_cm
  FROM players
 LIMIT 5;

 * sqlite:///./databases/imdb.db
Done.


heightFeet,foot_to_cm,heightInches,inch_to_cm
6,30.48,9,2.54
6,30.48,7,2.54
6,30.48,8,2.54
6,30.48,10,2.54
6,30.48,6,2.54


## 數值運算符

## 針對整數（INTEGER）與帶小數點的浮點數（REAL）可以使用數值運算符衍生計算欄位

![mutate](https://github.com/ccwu0918/classroom-hahow-sqlfifty/blob/main/04-calculated-fields/mutate.png?raw=1)

## 基礎數值運算符

|數值運算符|作用描述|
|--------|-------|
|`+`|相加|
|`-`|相減|
|`*`|相乘|
|`/`|相除|
|`%`|回傳餘數|
|`()`|優先運算|

## 運用數值運算符衍生以公分為單位的球員身高

In [11]:
%%sql
SELECT heightFeet,
       heightInches,
       heightFeet * 30.48 + heightInches * 2.54 AS heightCentimeters
  FROM players
 LIMIT 5;

 * sqlite:///./databases/imdb.db
Done.


heightFeet,heightInches,heightCentimeters
6,9,205.74
6,7,200.66
6,8,203.2
6,10,208.28
6,6,198.12


## 數值運算時要特別注意優先順序，例如先乘除後加減

\begin{equation}
Celsius_{(^{\circ} C)} = (Fahrenheit_{(^{\circ} F)} - 32) \times \frac{5}{9}
\end{equation}

In [12]:
%%sql
SELECT 212 - 32 * 5/9 AS wrong_conversion,      -- 錯誤的華氏轉攝氏
       (212 - 32) * 5/9 AS correct_conversion;  -- 正確的華氏轉攝氏

 * sqlite:///./databases/imdb.db
Done.


wrong_conversion,correct_conversion
195,100


## 在使用 `/` 相除兩個整數的時候要特別注意所衍生的欄位依然會以整數類型存在

In [13]:
%%sql
SELECT 1 / 2 AS an_integer;

 * sqlite:///./databases/imdb.db
Done.


an_integer
0


## 若希望衍生欄位是浮點數類型，要為分子或者分母加上小數點

In [15]:
%%sql
SELECT 1.0 / 2 AS a_real;

 * sqlite:///./databases/imdb.db
Done.


a_real
0.5


##  文字運算符

##  針對文字（TEXT）可以使用文字運算符衍生計算欄位

![mutate](https://github.com/ccwu0918/classroom-hahow-sqlfifty/blob/main/04-calculated-fields/mutate.png?raw=1)

## 基礎文字運算符

`||`：兩個垂直線（可透過 `Shift + \` 按出來）能夠連接文字。

In [16]:
%%sql
SELECT firstName,
       lastName,
       firstName || ' ' || lastName AS player_name
  FROM players
 LIMIT 5;

 * sqlite:///./databases/imdb.db
Done.


firstName,lastName,player_name
LeBron,James,LeBron James
Carmelo,Anthony,Carmelo Anthony
Udonis,Haslem,Udonis Haslem
Dwight,Howard,Dwight Howard
Andre,Iguodala,Andre Iguodala


## 重點統整

- `SELECT` 能夠單獨存在於一段 SQL 之中。
- 針對整數與帶小數點的浮點數可以使用**數值運算符**衍生計算欄位。
- 針對文字可以使用**文字運算符**衍生計算欄位。

```sql
/*
截至目前學起來的 SQL 有哪些？
SQL 寫作順序必須遵從標準 SQL 的規定。
*/
SELECT column_names  -- 選擇哪些欄位
  FROM table_name    -- 從哪個資料庫的資料表
 LIMIT m;            -- 查詢結果顯示前 m 列就好
```